In [1]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle


In [2]:
jan = pd.read_parquet('fhv_tripdata_2021-01.parquet')
feb = pd.read_parquet('fhv_tripdata_2021-02.parquet')

In [3]:
jan.shape

(1154112, 7)

Q1 -> 1154112

In [4]:
jan['duration_s']=(jan.dropOff_datetime-jan.pickup_datetime).dt.total_seconds()

In [5]:
(jan.duration_s).mean()/60

19.167224093791013

Q2 -> 19.16

In [6]:
jan['duration_minutes'] = jan.duration_s/60 

In [7]:
jan[(jan.duration_minutes>=1)&(jan.duration_minutes<=60)].shape

(1109826, 9)

In [8]:
jan.shape

(1154112, 9)

In [9]:
1154112 - 1109826

44286

Number of dropped records 44286

In [10]:
jan_final = jan[(jan.duration_minutes>=1)&(jan.duration_minutes<=60)]

In [11]:
jan_final.PUlocationID = jan_final.PUlocationID.fillna("-1")
jan_final.DOlocationID = jan_final.DOlocationID.fillna("-1")

C:\Users\Veera Vignesh\AppData\Local\Temp\ipykernel_32692\2809167046.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_final.PUlocationID = jan_final.PUlocationID.fillna("-1")
C:\Users\Veera Vignesh\AppData\Local\Temp\ipykernel_32692\2809167046.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_final.DOlocationID = jan_final.DOlocationID.fillna("-1")


In [12]:
jan_final.PUlocationID.value_counts()

-1       927008
221.0      8330
206.0      6797
129.0      5379
115.0      4082
          ...  
111.0         5
27.0          4
34.0          3
2.0           2
110.0         1
Name: PUlocationID, Length: 262, dtype: int64

In [13]:
927008/jan_final.shape[0]

0.8352732770722617

Q3 -> 83%

In [14]:
cat = ['PUlocationID','DOlocationID']
jan_final[cat] = jan_final[cat].astype('str')

C:\Users\Veera Vignesh\AppData\Local\Temp\ipykernel_32692\662191961.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_final[cat] = jan_final[cat].astype('str')


In [15]:
train_dicts = jan_final[cat].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [16]:
X_train.shape

(1109826, 525)

Q4 -> 525

In [17]:
target = 'duration_minutes'
y_train = jan_final[target].values

In [18]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519403716007

Q5 -> 10.52

In [19]:
def preprocess(df):
    df['duration_minutes']=(df.dropOff_datetime-df.pickup_datetime).dt.total_seconds()/60
    df = df[(df.duration_minutes>=1)&(df.duration_minutes<=60)]
    df.loc[:,'PUlocationID'] = df.PUlocationID.fillna("-1")
    df.loc[:,'DOlocationID'] = df.DOlocationID.fillna("-1")
    df[cat] = df[cat].astype('str')
    return df

In [20]:
val = preprocess(feb)

C:\Users\Veera Vignesh\AppData\Local\Temp\ipykernel_32692\1016185276.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'PUlocationID'] = df.PUlocationID.fillna("-1")
C:\Users\Veera Vignesh\AppData\Local\Temp\ipykernel_32692\1016185276.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'DOlocationID'] = df.DOlocationID.fillna("-1")
C:\Users\Veera Vignesh\AppData\Local\Temp\ipykernel_32692\1016185276.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [21]:
val_dicts = val[cat].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [22]:
X_val.shape

(990113, 525)

In [23]:
y_val = val['duration_minutes'].values

In [24]:
y_val_pred = lr.predict(X_val)

In [25]:
mean_squared_error(y_val, y_val_pred, squared=False)

11.014287719752

Q6 -> 11.01